In [1]:
import numpy as np
from scipy.stats import wilcoxon
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
model_names = ['Llama', 'rwkv', 'roberta-large', 'gpt2xl']

In [3]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for shuffle_str in ['shuffled', 'contig']:
        for noL2_str in ['_noL2custom', '']:
            results = pd.read_csv(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures_data/figure1/{dataset}_pearson_r{noL2_str}_{shuffle_str}.csv')
            for fe in ['', '-mp', '-sp']:
                results_gpt2xl = results.loc[results.Model==f'GPT2-XL{fe}']['perf'].to_numpy()
                results_oasm = results.loc[results.Model=='OASM']['perf'].to_numpy()
                result = wilcoxon(results_gpt2xl, results_oasm, alternative='greater')
                if np.mean(results_oasm) > np.mean(results_gpt2xl):
                    pass
                else:
                    store_pvalues[f"{dataset}_{shuffle_str}{noL2_str}_{fe}"] = f"GPT2XL better: {result.pvalue}"
store_pvalues

/home3/ebrahim2/miniconda3/envs/llm_brain_new/lib/python3.11/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)


{'pereira_contig_noL2custom_': 'GPT2XL better: 0.0009765625',
 'pereira_contig_noL2custom_-mp': 'GPT2XL better: 0.0009765625',
 'pereira_contig_noL2custom_-sp': 'GPT2XL better: 0.0009765625',
 'pereira_contig_': 'GPT2XL better: 0.0009765625',
 'pereira_contig_-mp': 'GPT2XL better: 0.0009765625',
 'pereira_contig_-sp': 'GPT2XL better: 0.0009765625',
 'blank_contig_noL2custom_': 'GPT2XL better: 0.033944577430914495',
 'blank_contig_noL2custom_-mp': 'GPT2XL better: 0.03125',
 'blank_contig_noL2custom_-sp': 'GPT2XL better: 0.03125',
 'blank_contig_': 'GPT2XL better: 0.0625',
 'blank_contig_-mp': 'GPT2XL better: 0.0625',
 'blank_contig_-sp': 'GPT2XL better: 0.0625',
 'fedorenko_contig_noL2custom_': 'GPT2XL better: 0.03125',
 'fedorenko_contig_noL2custom_-mp': 'GPT2XL better: 0.03125',
 'fedorenko_contig_noL2custom_-sp': 'GPT2XL better: 0.03125',
 'fedorenko_contig_': 'GPT2XL better: 0.03125',
 'fedorenko_contig_-mp': 'GPT2XL better: 0.03125',
 'fedorenko_contig_-sp': 'GPT2XL better: 0.03125

In [11]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for shuffle_str in ['shuffled', 'contig']:
        for noL2_str in ['_noL2custom', '']:
            results = pd.read_csv(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures_data/figure1/{dataset}_pearson_r{noL2_str}_{shuffle_str}.csv')
            for fe in ['', '-mp', '-sp']:
                results_gpt2xl = results.loc[results.Model==f'GPT2-XL{fe}']['perf'].to_numpy()
                results_oasm = results.loc[results.Model=='OASM']['perf'].to_numpy()
                result = wilcoxon(results_gpt2xl, results_oasm, alternative='greater')
                if np.mean(results_oasm) > np.mean(results_gpt2xl):
                    pass
                else:
                    store_pvalues[f"{dataset}_{shuffle_str}{noL2_str}_{fe}"] = f"GPT2XL better: {result.pvalue}"
store_pvalues

/home3/ebrahim2/miniconda3/envs/llm_brain/lib/python3.11/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)


{'pereira_contig_noL2custom_': 'GPT2XL better: 0.0009765625',
 'pereira_contig_noL2custom_-mp': 'GPT2XL better: 0.0009765625',
 'pereira_contig_noL2custom_-sp': 'GPT2XL better: 0.0009765625',
 'pereira_contig_': 'GPT2XL better: 0.0009765625',
 'pereira_contig_-mp': 'GPT2XL better: 0.0009765625',
 'pereira_contig_-sp': 'GPT2XL better: 0.0009765625',
 'blank_contig_noL2custom_': 'GPT2XL better: 0.033944577430914495',
 'blank_contig_noL2custom_-mp': 'GPT2XL better: 0.03125',
 'blank_contig_noL2custom_-sp': 'GPT2XL better: 0.03125',
 'blank_contig_': 'GPT2XL better: 0.0625',
 'blank_contig_-mp': 'GPT2XL better: 0.0625',
 'blank_contig_-sp': 'GPT2XL better: 0.0625',
 'fedorenko_contig_noL2custom_': 'GPT2XL better: 0.03125',
 'fedorenko_contig_noL2custom_-mp': 'GPT2XL better: 0.03125',
 'fedorenko_contig_noL2custom_-sp': 'GPT2XL better: 0.03125',
 'fedorenko_contig_': 'GPT2XL better: 0.03125',
 'fedorenko_contig_-mp': 'GPT2XL better: 0.03125',
 'fedorenko_contig_-sp': 'GPT2XL better: 0.03125

In [24]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for fe in ['-lt', '-mp', '-sp']:
        results = pd.read_csv(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures_data/figure4/{dataset}_pearson_r_gpt2xl.csv')
        results_gpt2xl = results.loc[results.Model==f'gpt2xl{fe}']['perf'].to_numpy()
        results_simple = results.loc[results.Model=='Simple']['perf'].to_numpy()
        result = wilcoxon(results_gpt2xl, results_simple, alternative='greater')
        
        if np.mean(results_simple) > np.mean(results_gpt2xl):
            store_pvalues[f"{dataset}_{fe}"] = f"Simple better: {result.pvalue}"
        else:
            store_pvalues[f"{dataset}_{fe}"] = f"GPT2XL better: {result.pvalue}"
            
store_pvalues
    

{'pereira_-lt': 'GPT2XL better: 0.001953125',
 'pereira_-mp': 'GPT2XL better: 0.052734375',
 'pereira_-sp': 'GPT2XL better: 0.009765625',
 'blank_-lt': 'Simple better: 1.0',
 'blank_-mp': 'Simple better: 1.0',
 'blank_-sp': 'Simple better: 1.0',
 'fedorenko_-lt': 'GPT2XL better: 0.59375',
 'fedorenko_-mp': 'GPT2XL better: 0.5',
 'fedorenko_-sp': 'GPT2XL better: 0.3125'}

In [8]:
store_pvalues = {}
for dataset in ['pereira', 'blank', 'fedorenko']:
    for fe in ['-lt', '-mp', '-sp']:
        results = pd.read_csv(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures_data/figure5/{dataset}_pearson_r.csv')
        results_gpt2xl = results.loc[results.Model==f'GPT2XLU{fe}']['perf'].to_numpy()
        results_simple = results.loc[results.Model=='Simple']['perf'].to_numpy()
        result = wilcoxon(results_gpt2xl, results_simple, alternative='greater')
        
        if np.mean(results_simple) > np.mean(results_gpt2xl):
            store_pvalues[f"{dataset}_{fe}"] = f"Simple better: {result.pvalue}"
        else:
            store_pvalues[f"{dataset}_{fe}"] = f"GPT2XL better: {result.pvalue}"
            
store_pvalues

{'pereira_-lt': 'Simple better: 1.0',
 'pereira_-mp': 'Simple better: 0.9755859375',
 'pereira_-sp': 'Simple better: 0.9345703125',
 'blank_-lt': 'Simple better: 1.0',
 'blank_-mp': 'Simple better: 0.96875',
 'blank_-sp': 'Simple better: 1.0',
 'fedorenko_-lt': 'Simple better: 0.90625',
 'fedorenko_-mp': 'Simple better: 0.84375',
 'fedorenko_-sp': 'Simple better: 0.90625'}

In [12]:
color_palette = sns.color_palette(["gray", "blue", 'black'])
def compute_frac_sig(feature_extraction_arr, dataset, figure_folder, llm_name='', ymax=30, chunked=False):
    
    if len(llm_name) > 0:
        llm_name = f"_{llm_name}"
        
    if chunked:
        df_name = 'blocked_pvalues'
    else:
        df_name = 'pvalues'
    
    df = pd.read_csv(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures_data/{figure_folder}/{df_name}_{dataset}{llm_name}.csv')
    fig, ax = plt.subplots(1,1,figsize=(3,4))
    plot_stats_df = {'values': [], 'feature_extraction_arr': [], 'dataset': []}
    
    for fe in feature_extraction_arr: 
        
        df_fe = df.loc[df.fe==fe].copy()
        print(df_fe.shape, dataset)
        df_fe['pval_sig'] = np.where(df_fe['pval']<0.05, 1, 0)

        df_fe['pval_LLM_sig'] = np.where(df_fe['pval_LLM_sig']<0.05, 1, 0)
  
        #df_fe = df_fe.loc[df_fe[f'pval_LLM_sig'] == 1]

        if df_fe.shape[0] == 0:
            print(f"No significant voxels/electrodes/fROIs for {llm_name}{fe} for {dataset}")
    
        # Calculate the mean proportion of significant p-values per subject
        subject_means = df_fe.groupby('subject')['pval_sig'].mean()

        plot_stats_df['values'].extend(np.array(subject_means)*100)
        plot_stats_df['feature_extraction_arr'].extend(np.repeat(fe, len(subject_means)))
        plot_stats_df['dataset'].extend(np.repeat(dataset, len(subject_means)))
        
        print(np.mean(subject_means))
        
        
    plot_stats_df = pd.DataFrame(plot_stats_df)
    sns.barplot(plot_stats_df, hue='feature_extraction_arr', y='values', x='dataset', alpha=0.4, legend=False, errorbar=None, palette=color_palette,ax=ax)
    sns.stripplot(plot_stats_df, hue='feature_extraction_arr', y='values', x='dataset', dodge=True, size=10, alpha=0.8, legend=False, palette=color_palette,ax=ax)
    sns.despine()
    ax.set_xlabel('')
    ax.set_xticklabels('')
    ax.set_xticks([])
    ax.set_yticks([0,ymax])
    ax.set_yticklabels([0,ymax], fontsize=20)
    ax.set_ylabel('')
    
    if chunked:
        print("Saving chunked")
        fig.savefig(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures/new_figures/{figure_folder}/sig/chunked_frac_sig_{dataset}_{llm_name}.png', bbox_inches='tight')
        fig.savefig(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures/new_figures/{figure_folder}/sig/chunked_frac_sig_{dataset}_{llm_name}.pdf', bbox_inches='tight')   
    else:
        fig.savefig(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures/new_figures/{figure_folder}/sig/frac_sig_{dataset}_{llm_name}.png', bbox_inches='tight')
        fig.savefig(f'/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures/new_figures/{figure_folder}/sig/frac_sig_{dataset}_{llm_name}.pdf', bbox_inches='tight')   
        
    plt.close()

In [13]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira', 'figure2', chunked=True)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko', 'figure2', chunked=True)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank', 'figure2', chunked=True)

(13553, 8) pereira
0.20226464850527032
(13553, 8) pereira
0.1672370054515032
(13553, 8) pereira
0.22631811863894144
Saving chunked
(97, 8) fedorenko
0.2136643026004728
(97, 8) fedorenko
0.28381796690307326
(97, 8) fedorenko
0.19347517730496452
Saving chunked
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.016666666666666666
Saving chunked


In [10]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira','figure2')
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko','figure2')
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank','figure2')

(13553, 8) pereira
0.22026116324437695
(13553, 8) pereira
0.18504614886382037
(13553, 8) pereira
0.2336163474787043
(97, 8) fedorenko
0.2938416075650118
(97, 8) fedorenko
0.28807328605200944
(97, 8) fedorenko
0.22160756501182033
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.016666666666666666


In [17]:
compute_frac_sig(['-lt'], 'pereira','synt', 'syntax', ymax=50)
compute_frac_sig(['-lt'], 'pereira','glove', 'glove', ymax=50)


(13553, 8) pereira
0.000662739322533137
(13553, 8) pereira
0.07619034767790638


/tmp/ipykernel_4052165/3388948640.py:37: UserWarning: The palette list has more values (3) than needed (1), which may not be intended.
  sns.barplot(plot_stats_df, hue='feature_extraction_arr', y='values', x='dataset', alpha=0.4, legend=False, errorbar=None, palette=color_palette,ax=ax)
/tmp/ipykernel_4052165/3388948640.py:38: UserWarning: The palette list has more values (3) than needed (1), which may not be intended.
  sns.stripplot(plot_stats_df, hue='feature_extraction_arr', y='values', x='dataset', dodge=True, size=10, alpha=0.8, legend=False, palette=color_palette,ax=ax)
/tmp/ipykernel_4052165/3388948640.py:37: UserWarning: The palette list has more values (3) than needed (1), which may not be intended.
  sns.barplot(plot_stats_df, hue='feature_extraction_arr', y='values', x='dataset', alpha=0.4, legend=False, errorbar=None, palette=color_palette,ax=ax)
/tmp/ipykernel_4052165/3388948640.py:38: UserWarning: The palette list has more values (3) than needed (1), which may not be int

In [14]:
for model in model_names:
    print(model)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira', 'figure4', model)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko', 'figure4', model)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'blank', 'figure4', model)

Llama
(13553, 8) pereira
0.007511045655375552
(13553, 8) pereira
0.024374079528718704
(13553, 8) pereira
0.0529501345526606
(97, 8) fedorenko
0.03333333333333333
(97, 8) fedorenko
0.06
(97, 8) fedorenko
0.04666666666666666
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.0
rwkv
(13553, 8) pereira
0.005522827687776141
(13553, 8) pereira
0.002430370977159241
(13553, 8) pereira
0.0246790398568884
(97, 8) fedorenko
0.024444444444444442
(97, 8) fedorenko
0.04666666666666666
(97, 8) fedorenko
0.06888888888888889
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.0
roberta-large
(13553, 8) pereira
7.363770250368188e-05
(13553, 8) pereira
0.04657064207450131
(13553, 8) pereira
0.04194303294805265
(97, 8) fedorenko
0.0
(97, 8) fedorenko
0.04666666666666666
(97, 8) fedorenko
0.057777777777777775
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.0
gpt2xl
(13553, 8) pereira
0.000589101620029455
(13553, 8) pereira
0.014311310339560169
(13553, 8) pereira
0.03411199099383911
(97, 8) fedorenko
0

In [17]:
for model in model_names:
    print(model)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira', 'figure4', model, chunked=True)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko', 'figure4', model, chunked=True)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'blank', 'figure4', model, chunked=True)

Llama


FileNotFoundError: [Errno 2] No such file or directory: '/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/figures_data/figure4/blocked_pvalues_pereira_Llama.csv'

for model in 
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira', 4, 'gpt2xl')
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko', 4, 'gpt2xl')
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank', 4, 'gpt2xl')

In [6]:
for model in model_names:
    print(model)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira', 'figure4', model)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko', 'figure4', model)
    compute_frac_sig(['-lt', '-mp', '-sp'], 'blank', 'figure4', model)

Llama
(13553, 8) pereira
0.007511045655375552
(13553, 8) pereira
0.024374079528718704
(13553, 8) pereira
0.0529501345526606
(97, 8) fedorenko
0.03333333333333333
(97, 8) fedorenko
0.06
(97, 8) fedorenko
0.04666666666666666
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.0
rwkv
(13553, 8) pereira
0.005522827687776141
(13553, 8) pereira
0.002430370977159241
(13553, 8) pereira
0.0246790398568884
(97, 8) fedorenko
0.024444444444444442
(97, 8) fedorenko
0.04666666666666666
(97, 8) fedorenko
0.06888888888888889
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.0
roberta-large
(13553, 8) pereira
7.363770250368188e-05
(13553, 8) pereira
0.04657064207450131
(13553, 8) pereira
0.04194303294805265
(97, 8) fedorenko
0.0
(97, 8) fedorenko
0.04666666666666666
(97, 8) fedorenko
0.057777777777777775
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.0
gpt2xl
(13553, 8) pereira
0.000589101620029455
(13553, 8) pereira
0.014311310339560169
(13553, 8) pereira
0.03411199099383911
(97, 8) fedorenko
0

In [8]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira','figure5')
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko','figure5')
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank','figure5')

(13553, 8) pereira
0.0
(13553, 8) pereira
0.0
(13553, 8) pereira
0.0
(97, 8) fedorenko
0.0
(97, 8) fedorenko
0.0
(97, 8) fedorenko
0.0
(60, 8) blank
0.0
(60, 8) blank
0.0
(60, 8) blank
0.0
